# <sapn style="color:#FFD167"> Outliers with WER & CER


<b><span style="color:#E27D07"> [ꆜ NEEDS TO START ꆜ] </span></b>

<b><span style="color:#4B92FF"> [❖ STARTED ❖] </span></b>

<b><span style="color:#50C878">[☆ DONE ☆]</span></b>

In [19]:
# Outliers Julie has reported
julies_outliers = [ "a02_snoerr.wav", "a02_gloer.wav", "a22_ny.wav", "a29_doer.wav", "a34_flue.wav", 
                "a34_lue.wav", "d02_bleie.wav", "d02_koe.wav", "d02_kvern.wav", "d02_snoe.wav", "d05_prins.wav", 
                "d08_bleie.wav", "d08_bror.wav", "d08_doer.wav", "d08_flagg.wav", "d08_kylling.wav", "d08_oere.wav", 
                "d09_bjoern.wav", "d09_kran.wav", "d15_loeve.wav"]

weird_files_from_julie = ["d02_bleie.wav", "d02_koe.wav", "d02_kvern.wav", # pause
                        "d05_prins.wav", # prins eller var det prins
                        "d08_bleie.wav", "d08_bror.wav", "d08_doer.wav", "d08_flagg.wav", "d08_kylling.wav", "d08_oere.wav",  # ekstra snakk i bakgrunnen
                        "d09_bjoern.wav", "d09_kran.wav", # mumbling av ting 
                        "d15_loeve.wav"] # pause 

exstra_talk =['d03_kort.wav']

# De andre outliersene til julie fant jeg ikke noe rart ved.

In [58]:
# 1) Find all the trascribed more than 1 word
# 2) Rule out one instances where the there is registered repetiction
# 3) Find Wer for the rest

import os
import self_made_functions as smf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import jiwer

directory = 'Transcriptions/'

# Loade 2 of the transcribed file - to compair - use medium-verbatim
df_fin, wv_path = smf.get_df_correct()
df_medium_verbatim_1 = pd.read_csv('./Transcriptions/transcriptions_nb-whisper-medium-verbatim_v1.csv')
# df_medium_verbatim_2 = pd.read_csv('./Transcriptions/transcriptions_nb-whisper-medium-verbatim_v2.csv')

hold_info_df_1 = pd.DataFrame(columns=['File name', 'WordCount', 'CER', 'WER', 'Rep'])
# hold_info_df_2 = pd.DataFrame(columns=['File name', 'WordCount', 'CER', 'WER', 'Rep'])

df_fin.columns.values

array(['File name', 'Score', 'Prosody', 'Noise/Disruption',
       'Pre-speech noise', 'Repetition', 'Word', 'Pronunciation',
       'pronScores', 'Assessor'], dtype=object)

In [64]:
df_fin[df_fin['pronScores']=='1 0 1 1 1']

,File name,Score,Prosody,Noise/Disruption,Pre-speech noise,Repetition,Word,Pronunciation,pronScores,Assessor
0,a01_artist.wav,4,NaN,NaN,NaN,NaN,artist,A t` I s t,1 0 1 1 1,Anne Marte
10,a01_boerste.wav,3,NaN,NaN,NaN,NaN,børste,b 9 s` t` @,1 0 1 1 1,Anne Marte
33,a01_flette.wav,3,1.0,NaN,NaN,NaN,flette,f l E t @,1 0 1 1 1,Anne Marte
84,a01_krone.wav,3,1.0,NaN,NaN,NaN,krone,k r u: n @,1 0 1 1 1,Anne Marte
86,a01_krykke.wav,3,NaN,NaN,NaN,NaN,krykke,k r Y k @,1 0 1 1 1,Anne Marte
...,...,...,...,...,...,...,...,...,...,...
8510,d14_trylle.wav,3,NaN,1.0,NaN,NaN,trylle,t r Y l @,1 0 1 1 1,Anne Marte
9198,d19_drikke.wav,3,NaN,NaN,NaN,NaN,drikke,d r I k @,1 0 1 1 1,Anne Marte
9225,d19_grave.wav,3,NaN,NaN,NaN,NaN,grave,g r A: v @,1 0 1 1 1,Jeanett
9253,d19_krykke.wav,4,NaN,NaN,NaN,NaN,krykke,k r Y k @,1 0 1 1 1,Anne Marte


In [66]:
# Start testing for first file
# Calculate WER, and add it with the wordcount, filename, CER and repetition in the new df 

for idx, row in df_medium_verbatim_1.iterrows():
    if pd.notna(row['Word']) and pd.notna(row['Transcribed']):
        # Add values to hold_info_df
        hold_info_df_1.loc[idx, 'WER'] = jiwer.wer(row['Word'], row['Transcribed'])
        hold_info_df_1.loc[idx, 'WordCount'] = len(row['Transcribed'].split())
        
        hold_info_df_1.loc[idx, 'File name'] = row['File name']
        hold_info_df_1.loc[idx, 'CER'] = row['CER (Character Error Rate)']
        hold_info_df_1.loc[idx, 'Rep'] = df_fin[df_fin['File name']==row['File name']]['Repetition'].values[0]
        hold_info_df_1.loc[idx, 'Score'] = row['OG Score']
        hold_info_df_1.loc[idx, 'Transcribed'] = row['Transcribed']
        hold_info_df_1.loc[idx, 'pronScores'] = df_fin['pronScores'][df_fin['File name']==row['File name']].values[0]
        
    else: # Reassure it is no empty transcriptions
        print(f'Error in row {idx} in df_medium_verbatim_1 {row}')
        #MT: a29_rund.wav - a25_troll.wav


In [67]:
hold_info_df_1

,File name,WordCount,CER,WER,Rep,Score,Transcribed,pronScores
0,a06_hylle.wav,1,0.25,1.0,NaN,1.0,hyll,1 0 1 0
1,a12_hylle.wav,1,0.0,0.0,1.0,5.0,hylle,1 1 1 1
2,a33_hylle.wav,1,0.0,0.0,NaN,5.0,hylle,1 1 1 1
3,d18_hylle.wav,1,0.0,0.0,NaN,5.0,hylle,1 1 1 1
4,a27_hylle.wav,1,0.571429,1.0,NaN,3.0,kylling,0 1 1 1
...,...,...,...,...,...,...,...,...
9311,d16_smart.wav,1,0.2,1.0,NaN,4.0,snart,0 1 1 1
9312,a10_smart.wav,1,0.571429,1.0,NaN,2.0,schmach,0 1 1 0
9313,a04_smart.wav,1,0.0,0.0,NaN,4.0,smart,1 1 0 1
9314,a25_smart.wav,1,0.2,1.0,NaN,4.0,snart,1 1 0 1


## How does my model transcrie the files Julie has provided as outliers? <b><span style="color:#50C878">[☆ DONE ☆]</span></b>

In [68]:
# Transcriptions for all julies outliers
hold_info_df_1[hold_info_df_1['File name'].isin(julies_outliers)]

,File name,WordCount,CER,WER,Rep,Score,Transcribed,pronScores
623,d08_bror.wav,3,0.826087,3.0,NaN,5.0,stor egentlig storebror,1 1 1 1
661,d02_kvern.wav,5,0.826087,5.0,NaN,5.0,kvelden kan vi ta pause,1 1 1 1
938,d08_bleie.wav,1,1.0,1.0,NaN,3.0,nei,1 1 0 1
959,d02_bleie.wav,6,0.807692,5.0,NaN,5.0,bleie kan vi ta litt pause,1 1 1 1
1225,d08_flagg.wav,6,0.875,6.0,NaN,5.0,ja det var det jeg tegna,1 1 1 1
1835,d08_oere.wav,1,0.5,1.0,NaN,3.0,mere,0 1 1
2432,a34_flue.wav,1,0.25,1.0,NaN,3.0,frue,1 0 1 1
3431,a29_doer.wav,1,0.333333,1.0,NaN,3.0,død,1 1 0
3450,d08_doer.wav,1,3.0,1.0,NaN,4.0,æ,1 1 1
3560,d15_loeve.wav,5,0.809524,4.0,NaN,4.0,løve kan jeg få pause,1 1 1 1


In [23]:
# Transcriptions for the ones i listen too and registerd more talking
hold_info_df_1[hold_info_df_1['File name'].isin(weird_files_from_julie)]

,File name,WordCount,CER,WER,Rep,Score,Transcribed
623,d08_bror.wav,3,0.826087,3.0,NaN,5.0,stor egentlig storebror
661,d02_kvern.wav,5,0.826087,5.0,NaN,5.0,kvelden kan vi ta pause
938,d08_bleie.wav,1,1.0,1.0,NaN,3.0,nei
959,d02_bleie.wav,6,0.807692,5.0,NaN,5.0,bleie kan vi ta litt pause
1225,d08_flagg.wav,6,0.875,6.0,NaN,5.0,ja det var det jeg tegna
1835,d08_oere.wav,1,0.5,1.0,NaN,3.0,mere
3450,d08_doer.wav,1,3.0,1.0,NaN,4.0,æ
3560,d15_loeve.wav,5,0.809524,4.0,NaN,4.0,løve kan jeg få pause
5071,d09_bjoern.wav,5,0.884615,5.0,NaN,2.0,bystadsa lærte seg å nisse
7443,d09_kran.wav,2,0.666667,2.0,1.0,5.0,gran gran


### Find files where it transcribed the word "pause" <b><span style="color:#50C878">[☆ DONE ☆]</span></b>

audio files where the model transcribed the children asking for a break: ['d02_kvern.wav', 'd02_bleie.wav', 'd15_loeve.wav', 'd03_kort.wav', 'd02_koe.wav'].

Most of these where in Julies findings except <b><span style="color:#4B92FF">d03_kort.wav </span></b>. The model transcribe the correct word, and the request for a break.

In [24]:
look_for = "pause"
pause_files = hold_info_df_1[hold_info_df_1['Transcribed'].str.contains(look_for)]
pause_files
# pause_files['File name'].values

,File name,WordCount,CER,WER,Rep,Score,Transcribed
661,d02_kvern.wav,5,0.826087,5.0,NaN,5.0,kvelden kan vi ta pause
959,d02_bleie.wav,6,0.807692,5.0,NaN,5.0,bleie kan vi ta litt pause
3560,d15_loeve.wav,5,0.809524,4.0,NaN,4.0,løve kan jeg få pause
5287,d03_kort.wav,2,0.6,1.0,NaN,5.0,kort pause
7743,d02_koe.wav,5,0.888889,4.0,NaN,5.0,kø kan vi ta pause


# Audio with repetition <b><span style="color:#4B92FF"> [❖ STARTED ❖] </span></b>

Find where there is repetition in the model and where the model has transcribed two words. 

It was assumed that this would rule out some of the sound files. That was not the case, looking at <b><span style="color:#4B92FF">"d09_kran.wav"</span></b> found by Julie. This file had repetition but also some noise and mumbling at the start and end of the sound clip. The medium-verbatim model transcribed the same word twice, but the wrong phonemes were transcribed.


The assumption that for the instances where two words were transcribed and repetitions of the word were registered, this could rule out outliers was wrong. Therefore, it was decided to look further into the transcriptions of the audio files with repetition in them. The test looked at the distribution of the files in each WER category. In most cases (24 out of 48), the model only transcribed one word, not registering the second word uttered. It was therefore checked if the scores for the audio files with repetition were low or the audio files with only one in case this could be why the model did not transcribe the second word. This was not the case since most scored 3 or 5.


Looking further upon the distribution of files for WER, 3 cases with WER = 3.0 and, in one case, WER = 4.0 were found. These files were checked to see if they had additional speech that was not the word repeated. 

When the audio was checked, the child started to say the word, stopping and saying the word. The transcriptions from the model are not representable.

An example here is the file  <b><span style="color:#4B92FF">"d01_plystre.wav"</span></b>; it has only a slight sound at the start, then the word. The "random" words are a showcase of hallucination.



In [25]:
# Find the instances where the repetion = 1 and word transcribed = 2
filtered_df = hold_info_df_1[(hold_info_df_1['WordCount'] == 2) & (hold_info_df_1['Rep'] == 1.0)]

transcribed = df_medium_verbatim_1[df_medium_verbatim_1['File name'].isin(filtered_df['File name'])]
print(f'Instances where the modeltranscribed 2 words and the word was repeated: {len(transcribed)}')
print(f'Same as in julies_outliers: {len(transcribed[transcribed['File name'].isin(julies_outliers)])}')

Instances where the modeltranscribed 2 words and the word was repeated: 29
Same as in julies_outliers: 1


In [26]:
# The infromation about the outlier Julie found
in_outliers = transcribed[transcribed['File name'].isin(julies_outliers)]['File name'].values[0]
df_fin[df_fin['File name']==in_outliers]

,File name,Score,Prosody,Noise/Disruption,Pre-speech noise,Repetition,Word,Pronunciation,pronScores,Assessor
7613,d09_kran.wav,5,NaN,1.0,NaN,1.0,kran,k r A: n,1 1 1 1,Jeanett


In [27]:
# Number of transcribed words for the ones with receptions in the sound file
rep_df = hold_info_df_1[hold_info_df_1['Rep'] == 1.0]
rep_df

# How may in each WER category is there
print("Instances of files with repetition=1 for each WER value",rep_df['WER'].value_counts(),'\n')

# What is the Score distribution fro the words whit repetition?
# Theory was that the words with repetition and lower score would make it harder for the model to transcribe them
rep_df_fin = df_fin[df_fin['Repetition'] == 1.0]
print("Compare if rep_df and rep_df_fin is the same length: ", len(rep_df)==len(rep_df_fin),'\n')
print('The number of instances for each "Score" for the audio files with repetition:', rep_df_fin['Score'].value_counts(),'\n')

# What is the Score distribution for the words with repetition and WER = 1.0 

re_df_wer1 = rep_df[rep_df['WER'] == 1.0]
print('Rep = 1 & WER = 1.0: The number for each "Score"', re_df_wer1['Score'].value_counts(),'\n')



Instances of files with repetition=1 for each WER value WER
1.0    24
2.0    15
0.0     5
3.0     3
4.0     1
Name: count, dtype: int64 

Compare if rep_df and rep_df_fin is the same length:  True 

The number of instances for each "Score" for the audio files with repetition: Score
3    17
5    13
2     9
4     9
Name: count, dtype: int64 

Rep = 1 & WER = 1.0: The number for each "Score" Score
3.0    9
5.0    7
4.0    5
2.0    3
Name: count, dtype: int64 



In [28]:
high_wer = rep_df[rep_df['WER'] > 2.0]
print('Rep = 1 & WER = 1.0: The number for each "Score"', high_wer['Score'].value_counts(),'\n')

Rep = 1 & WER = 1.0: The number for each "Score" Score
2.0    2
3.0    1
5.0    1
Name: count, dtype: int64 



In [50]:
print('The high WER, (WER > 2.0), files with repetition:')# , high_wer['File name'].values)
high_wer
# d01_artist.wav <- not realy teh repetion of the word - can't realy hear the child say artist 

The high WER, (WER > 2.0), files with repetition:


,File name,WordCount,CER,WER,Rep,Score,Transcribed
62,d01_artist.wav,4,0.823529,4.0,1.0,2.0,og til og tirsdag
1976,a27_sverd.wav,3,0.785714,3.0,1.0,2.0,svært og svært
5320,d18_fortau.wav,3,0.692308,3.0,1.0,3.0,fort fort der
7942,d01_plystre.wav,3,0.65,3.0,1.0,5.0,hvorfor plystrer det


In [51]:
print("Sum of the high wer outliers in julies outliers =", high_wer['File name'].isin(julies_outliers).sum())

Sum of the high wer outliers in julies outliers = 0


The audio files have some sounds, or the child is starting to say the word, stopping and saying the word. The transcriptions are not representable. 

The transcriptions are wrong. <b><span style="color:#4B92FF"> "d01_plystre.wav " </b></span>  sounds like it only has a slight sound at the start, then the word. The "random" words are a showcase of hallucination.

--------------- SE om en av dis filer !!! -----------------------------------------------------------


# Next -> Hvor mange med 2 ord er rpetion, og hvor amnge med 2 ord har ikke reppetion!! <b><span style="color:#E27D07"> [ꆜ NEEDS TO START ꆜ] </span></b>

## Make script that saves specific audio files to a local script  <b><span style="color:#50C878">[☆ DONE ☆]</span></b>



In [41]:
# Save the audio files to a local directory to listen to
from pydub import AudioSegment
# Put specific sound file in a new direcory so it can be listen to

def save_audio(audio:np.array, name:str, directory:str):
    # Save the audio file
    directory = os.path.join(directory, name)
    # print(f'Saving audio file: {directory}')
    audio.export(directory, format="wav")
    return directory

def save_intrestin_audio_local(file_name:str, wv_path:str):
    # file_name = 'd01_artist.wav'
    audio_path = os.path.join(wv_path, file_name)
    audio = AudioSegment.from_file(audio_path)

    audio_dir = './Audio_files_intresting'
    print(audio_path)

    save_audio(audio, file_name, audio_dir)

# for file_name in julies_outliers:# julies_outliers:high_wer['File name']:
#     save_intrestin_audio_local(file_name, wv_path)


for file_name in high_wer['File name']:# julies_outliers:high_wer['File name']:
    save_intrestin_audio_local(file_name, wv_path)



/talebase/data/speech_raw/teflon_no/speech16khz/d01_artist.wav
/talebase/data/speech_raw/teflon_no/speech16khz/a27_sverd.wav
/talebase/data/speech_raw/teflon_no/speech16khz/d18_fortau.wav
/talebase/data/speech_raw/teflon_no/speech16khz/d01_plystre.wav


In [31]:
save_file = 'd03_kort.wav'	

save_intrestin_audio_local(save_file, wv_path)

/talebase/data/speech_raw/teflon_no/speech16khz/d03_kort.wav


<b>The result</b> for the transciptions of the audio files where it was an repetion of the word was for most of them 24 instances the model only transcribed 1 word.

## Look at the CER and WER for the other files. <b><span style="color:#4B92FF"> [❖ STARTED ❖] </span></b>

<b><span style="color:#E27D07"> [ꆜ NEEDS TO START ꆜ] </span></b>
<b><span style="color:#4B92FF"> [❖ STARTED ❖] </span></b>
<b><span style="color:#50C878">[☆ DONE ☆]</span></b>

In [32]:
high_wer_df1 = hold_info_df_1[hold_info_df_1['WER'] > 2.0]
high_wer_df1

,File name,WordCount,CER,WER,Rep,Score,Transcribed
58,a07_artist.wav,4,0.705882,4.0,NaN,2.0,hørte jeg ikke st
60,a02_artist.wav,3,0.714286,3.0,NaN,2.0,er dette tørst
62,d01_artist.wav,4,0.823529,4.0,1.0,2.0,og til og tirsdag
82,a12_artist.wav,3,0.666667,3.0,NaN,2.0,er ikke lyst
94,a27_artist.wav,3,0.545455,3.0,NaN,4.0,er det tist
...,...,...,...,...,...,...,...
9224,a10_glad.wav,4,0.857143,4.0,NaN,2.0,hva tror du da
9235,a01_glad.wav,4,0.933333,4.0,NaN,3.0,hvor vil jeg ha
9250,a30_glad.wav,3,0.818182,3.0,NaN,2.0,hva er det?
9260,d09_glad.wav,4,0.769231,4.0,NaN,3.0,du går bra da


In [33]:
hold_info_df_1.sort_values(by='WER', ascending=False)
hold_info_df_1.value_counts()

File name            WordCount  CER       WER  Rep  Score  Transcribed         
a02_flaske.wav       1          0.500000  1.0  1.0  3.0    sveske                  1
a03_flakse.wav       2          0.444444  2.0  1.0  3.0    fa frakse               1
a31_pute.wav         2          0.428571  1.0  1.0  4.0    du pute                 1
a31_skitten.wav      2          0.400000  2.0  1.0  3.0    ski sliten              1
a32_garn.wav         2          0.714286  2.0  1.0  4.0    gla gal                 1
a33_bleie.wav        2          0.500000  1.0  1.0  5.0    blei bleie              1
a33_fortau.wav       2          0.454545  1.0  1.0  5.0    fort fortau             1
a33_kurv.wav         2          0.666667  2.0  1.0  3.0    kurf kurf               1
d01_artist.wav       4          0.823529  4.0  1.0  2.0    og til og tirsdag       1
d01_fortid.wav       2          0.538462  1.0  1.0  5.0    fortid fortid           1
d01_plystre.wav      3          0.650000  3.0  1.0  5.0    hvorfor ply

## <span style="color:#E45653">Scoring repiting words:</span> <span style="color:#E27D07"> [ꆜ NEEDS TO START ꆜ] </span> 

For some of the trascriptions where the repettion are the model manages to transcribe the second, word right, og it has two right transcriptions.

<span style="color:#E27D07"><b> THIS NEEDS TO BE DELT WITH!! </b></span>

In [34]:
# File name            WordCount  CER       WER  Rep  Score  Transcribed
# a31_pute.wav         2          0.428571  1.0  1.0  4.0    du pute  
# d04_soester.wav      2          0.400000  1.0  1.0  5.0    søs søster  
# d10_artist.wav       2          0.500000  1.0  1.0  3.0    hetti artist
# d10_smykke.wav       2          0.400000  1.0  1.0  3.0    smy smykke              
# d16_bryter.wav       2          0.333333  1.0  1.0  5.0    ly bryter
# a27_sjakk.wav        2          0.545455  1.0  1.0  4.0    sjakk sjakk

## Look closer and listen to the soundfiles wiht high WER <span style="color:#4B92FF"> [❖ STARTED ❖] </span>

